In [29]:
from uncertainties import ufloat, umath
from uncertainties import unumpy as unp
import numpy as np
from matplotlib import pyplot as plt
from scipy.odr import ODR,Model,RealData
import sys

In [30]:
#paar Unsicherheitsfunktionen
def rechteck(x): #Unsicherheit bei Rechtecksverteilung oder bei zwei Dreiecksverteilungen
    return x/(2*np.sqrt(3))

def dreieck(x): #Unsicherheit bei Dreiecksverteilung
    return x/(2*np.sqrt(6))

def kombUnsicherheit(x,u_mess_rel,offset_u_mess,u_ablese): #Kombination aus Unsicherheit des Messgerätes und Ableseunsicherheit
    u_mess=(x+offset_u_mess)*u_mess_rel
    return np.sqrt(u_mess**2+u_ablese**2)

def rel_abweichung(messwert,theoriewert):
  return (messwert-theoriewert)/theoriewert

def rel_unsicherheit(x):
  return x.s/x.n

def mittelwert(uarray):
  t_factor=np.array([1.84,1.32,1.2,1.14,1.11,1.09,1.08,1.07,1.06,1.05])
  length=uarray.size
  mean=uarray.mean()
  if (length<=10):
    mean=ufloat(mean.n,mean.s*t_factor[length-1])
  return mean

def uprint(ufloat):
  print('{:.2uS}'.format(ufloat))




#prints "VARIABLE_NAME:   VARIABLE"
#experimentell: gibt nicht den namen der eingegebenen variable zurück, 
# sondern den variablennamen der als erstes in der globalen Umgebung dem Objekt zugeteilt wurde
def uprint_name(ufloat):
   name = [k for k, v in globals().items() if v is ufloat][0]
   print(name, end=":   ")
   uprint(ufloat)

def uarray_print_name(uarray):
   name = [k for k, v in globals().items() if v is uarray][0]
   print(name, ":")
   uarray_print(uarray)

def uarray_print(uarray):
  for ufloat in uarray:
    uprint(ufloat)

# 4. Prisma untersuchen
## Winkel der minimalen Laserablenkung bestimmen und daraus den Brechungsindex von Flintglas berechnen
Messwerte

In [44]:
u_rel_wellenlaenge_laser_blau=0.05
u_wellenlaenge_laser_rot = dreieck(60e-9)

wellenlaenge_blau=ufloat(405e-9,405e-9*u_rel_wellenlaenge_laser_blau)
wellenlaenge_rot = ufloat(650e-9,u_wellenlaenge_laser_rot)

u_ankathete=dreieck(0.004)
ankathete_blau = ufloat(0.16,u_ankathete)
uprint_name(ankathete_blau)
ankathete_rot = ufloat(0.16,u_ankathete)

u_gegenkathete = dreieck(0.008)
gegenkathete_blau = ufloat(0.2,u_gegenkathete)
uprint_name(gegenkathete_blau)
gegenkathete_rot = ufloat(0.165,u_gegenkathete)
uprint_name(gegenkathete_rot)

apexwinkel = unp.radians(ufloat(60,60*0.01))
uprint(unp.degrees(apexwinkel))

ankathete_blau:   0.16000(82)
gegenkathete_blau:   0.2000(16)
gegenkathete_rot:   0.1650(16)
60.00(60)


In [32]:
def n_glas(ausfallswinkel,apexwinkel):
    return unp.sin((ausfallswinkel+apexwinkel)/2)/unp.sin(apexwinkel/2)


In [33]:
ausfallswinkel_blau = unp.arctan(gegenkathete_blau/ankathete_blau)
ausfallswinkel_rot=unp.arctan(gegenkathete_rot/ankathete_rot)
uprint(unp.degrees(ausfallswinkel_blau))
uprint(unp.degrees(ausfallswinkel_rot))

n_glas_blau = n_glas(ausfallswinkel_blau,apexwinkel)
n_glas_rot = n_glas (ausfallswinkel_rot,apexwinkel)
uprint_name(n_glas_blau)
uprint_name(n_glas_rot)

51.34(27)
45.88(32)
n_glas_blau:   1.6516(95)
n_glas_rot:   1.5960(88)


# 5. Halbkreisküvette
### Winkel der Beugungsmaxima von Luft und destilliertem Wasser bestimmen, daraus Brechungsindex von Wasser berechnen
Messwerte

In [46]:
gitterkonstante = ufloat(600e3,600e3*0.01)

u_winkel = dreieck(2)
winkel_luft_blau = ufloat(14,u_winkel)
winkel_luft_rot = ufloat(24,u_winkel)
winkel_wasser_blau=ufloat(11,u_winkel)
winkel_wasser_rot = ufloat(18,u_winkel)

uprint_name(winkel_luft_blau)
uprint_name(winkel_luft_rot)
uprint_name(winkel_wasser_blau)
uprint_name(winkel_wasser_rot)


winkel_luft_blau:   14.00(41)
winkel_luft_rot:   24.00(41)
winkel_wasser_blau:   11.00(41)
winkel_wasser_rot:   18.00(41)


In [35]:
def n_wasser(winkel_luft,winkel_wasser):
    return unp.sin(unp.radians(winkel_luft))/unp.sin(unp.radians(winkel_wasser))

In [48]:
n_wasser_blau = n_wasser(winkel_luft_blau,winkel_wasser_blau)
n_wasser_rot = n_wasser(winkel_luft=winkel_luft_rot,winkel_wasser=winkel_wasser_rot)

rel_abweichung_n_wasser_blau = rel_abweichung(n_wasser_blau,1.34315)
rel_abweichung_n_wasser_rot = rel_abweichung(n_wasser_rot,1.33211)

uprint_name(n_wasser_blau)
uprint_name(n_wasser_rot)

uprint_name(rel_abweichung_n_wasser_blau)
uprint_name(rel_abweichung_n_wasser_rot)

n_wasser_blau:   1.268(59)
n_wasser_rot:   1.316(36)
rel_abweichung_n_wasser_blau:   -0.056(44)
rel_abweichung_n_wasser_rot:   -0.012(27)


Theoriewinkel für Luft

In [37]:
winkel_luft_blau_theo = unp.degrees(unp.arcsin(gitterkonstante*wellenlaenge_blau))
winkel_luft_rot_theo = unp.degrees(unp.arcsin(gitterkonstante*wellenlaenge_rot))

uprint_name(winkel_luft_blau_theo)
uprint_name(winkel_luft_rot_theo)

winkel_luft_blau_theo:   14.06(73)
winkel_luft_rot_theo:   22.95(52)


# 6. Brennweite der Sammellinse bestimmen
Messwerte

In [49]:
brennweite_sammellinse = ufloat(0.087,dreieck(6e-3))
uprint_name(brennweite_sammellinse)

brennweite_sammellinse:   0.0870(12)


# 8. Brennweite der Streulinse bestimmen
Messwerte

In [50]:
dist_linsen = ufloat(0.056,dreieck(0.01))
uprint_name(dist_linsen)

dist_linsen:   0.0560(20)


In [40]:
brennweite_zerstreuungslinse = dist_linsen - brennweite_sammellinse
uprint_name(brennweite_zerstreuungslinse)

brennweite_zerstreuungslinse:   -0.0310(24)
